In [ ]:
!pip install geojson
!pip install geopandas
import pandas as pd
from datetime import date
import zipfile
import json
from geojson import Feature, FeatureCollection, Point
import geopandas
from google.colab import files

     |████████████████████████████████| 1.0 MB 5.5 MB/s 
     |████████████████████████████████| 16.7 MB 44.0 MB/s 
     |████████████████████████████████| 6.3 MB 28.0 MB/s 


In [ ]:
# UPLOAD SITE DATA REPORT

site_data_report = files.upload()

Saving MyROR Site Data Report 2022-02-10.xlsx to MyROR Site Data Report 2022-02-10.xlsx


In [ ]:
filename = next(iter(site_data_report))
df = pd.read_excel(filename)

# Select high level active only sites
high_level_active = ['Active', 'Active - Non-Reporting', 'Slated for deactivation', 'COVID-19 Freeze'] 
rslt_df = df[df['Status'].isin(high_level_active)] 

In [ ]:
# UPLOAD AFFILIATE CONTACT LIST

affiliate_contact_list = files.upload()

Saving 2022-01-18_Find-a-Site_Affiliate_Contact_Information.xlsx to 2022-01-18_Find-a-Site_Affiliate_Contact_Information.xlsx


In [ ]:
affiliate_filename = next(iter(affiliate_contact_list))

affiliate_contact_df = pd.read_excel(affiliate_filename)

rslt_df = rslt_df.join(affiliate_contact_df.set_index('Affiliate Name'), on='Affiliate Name')
rslt_df = rslt_df.set_index('Site ID')

contact_list1 = []
contact_list2 = []
contact_list3 = []
for ind, row in rslt_df.iterrows():
  email_check1 = rslt_df['Primary Contact Email'][ind]
  email_check2 = rslt_df['Medical Consultant Email'][ind]
  email_check3 = rslt_df['Affiliate Contact'][ind]

  # If there is NO  Primary Contact Email, substitue Medical Consultant Email for Primary Contact Email
  if type(email_check1) == float and type(email_check2) == str:
    rslt_df.loc[ind, "Primary Contact Email"] = rslt_df.loc[ind, "Medical Consultant Email"]
    contact_list1.append(ind)
  # If there is NO Primary Contcat Email AND NO medical contact email, substitude info@reachoutandread.org
  elif type(email_check1) == float and type(email_check2) == float and type(email_check3) ==str:
    rslt_df.loc[ind, "Primary Contact Email"] = "info@reachoutandread.org"
    contact_list2.append(ind)
  # If there is NO Primary Contact Email AND NO medical contact email AND NO affiliate contact email, substitude info@reachoutandread.org
  elif type(email_check1) == float and type(email_check2) == float and ((type(email_check3) ==float) or email_check3 == "NONE"):
    rslt_df.loc[ind, "Primary Contact Email"] = "info@reachoutandread.org"
    contact_list3.append(ind)

rslt_df = rslt_df.reset_index(level=['Site ID'])
rslt_df = rslt_df[['Site ID', 'Site Name', 'Affiliate Contact','Physical Location: Line 1', 'Physical Location: Line 2', 'Physical Location: Line 3', 'Physical Location: City', 'Physical Location: State', 'Physical Location: Zip', 'Latitude', 'Longitude', 'Site Interest: potential volunteers', 'Site Interest: new or used book donations', 'Primary Contact Email']]
rslt_df = rslt_df.rename(columns = {'Site ID': 'site_id', 'Site Name': 'sitename', 'Affiliate Contact': 'affiliate_email','Physical Location: Line 1': 'physical_1', 'Physical Location: Line 2':'physical_2', 'Physical Location: Line 3': 'physical_3', 'Physical Location: City': 'city', 'Physical Location: State' : 'state', 'Physical Location: Zip': 'zip', 'Site Interest: potential volunteers': 'site_volunteers', 'Site Interest: new or used book donations': 'site_donations', 'Primary Contact Email': 'primary_email'})
rslt_df = rslt_df.round({'Latitude': 4, 'Longitude': 4})
rslt_df = rslt_df.fillna("")

for ind, row in rslt_df.iterrows():
  if type(rslt_df.loc[ind, "site_volunteers"]) == float:
    rslt_df.loc[ind, "site_volunteers"] = "N"
  
  if type(rslt_df.loc[ind, "site_donations"]) == float:
    rslt_df.loc[ind, "site_donations"] = "N"

if (rslt_df['primary_email'].isna().any()) == True:
  print("Error with emails")
else: print("All is well with emails!")

if (rslt_df['site_volunteers'].isna().any()) == True:
  print("Error with Site Volunteers")
else:
  print("All is well with volunteers!")

if (rslt_df['site_donations'].isna().any()) == True:
  print("Error with Site Donations")
else:
  print("All is well with donations!")

# CREATE EXCEL SPREADSHEET WITH MISSING CONTACT INFO

contact_df = pd.DataFrame({'Missing Primary Contact': pd.Series(contact_list1), 'Missing Primary Contact and Medical Contact': pd.Series(contact_list2), 'Missing Primary Contact, Medical Contact, and Affiliate Contact': pd.Series(contact_list3)})
today = str(date.today())
contact_filename = "Missing_Contact_List_" + today + ".xlsx"
contact_df.to_excel(contact_filename, index=False)

# SAVE RESULTS DATAFRAME AS EXCEL FILE 
today = str(date.today())
excel_filename = "find-a-site_" + today + ".xlsx"
rslt_df.to_excel(excel_filename, index=False)

# CREATE CSV UTF-8 FILE FOR "LARGE" JSON FILE
large_csv = "find-a-site_" + today + "_large" + ".csv"
rslt_df.to_csv(large_csv,encoding='utf-8-sig', index=False)

# CREATE JSON.TXT FILE
json_dict = rslt_df.to_json(orient="records")
parsed = json.loads(json_dict)
#json_object = json.dumps(parsed)
json_object = json.dumps(parsed, indent=2, sort_keys=True)
json_filename = "find-a-site_" + today + "_large_.json.txt"
with open(json_filename, "w") as outfile:
    outfile.write(json_object)

# CREATE CSV UTF-8 FILE FOR "SMALL" JSON FILE
small_df = rslt_df[['site_id', 'Latitude', 'Longitude']]
small_csv = "find-a-site_" + today + "_small" + ".csv"
small_df.to_csv(small_csv,encoding='utf-8-sig', index=False)

# CREATE GEOJSON FILE
geo_small_df = geopandas.GeoDataFrame(
    small_df, geometry=geopandas.points_from_xy(small_df['Longitude'], small_df['Latitude']))
geo_small_df.to_json()
geojson_filename =  "find-a-site_" + today + "_small_.geojson.txt"
geo_small_df.to_file(geojson_filename, driver="GeoJSON") 

# ZIP FILES TOGETHER INTO FOLDER
list_files = [contact_filename, excel_filename, large_csv, small_csv, json_filename, geojson_filename]
# list_files = [contact_filename, json_filename, geojson_filename]

folder_name = "Find-A-Site_" + today + ".zip"
with zipfile.ZipFile(folder_name, 'w') as zipF:
  for file in list_files:
    zipF.write(file, compress_type=zipfile.ZIP_DEFLATED)

files.download(folder_name)
  


All is well with emails!
All is well with volunteers!
All is well with donations!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>